PUBG telemetry events: https://documentation.pubg.com/en/telemetry-events.html
<br>telemetry library github: https://github.com/crflynn/chicken-dinner/tree/master/chicken_dinner/models/telemetry

## Match log data
import library

In [1]:
## for calling dataset
import requests

## parsing dataset
from chicken_dinner.pubgapi import PUBG
from chicken_dinner.constants import COLORS
from chicken_dinner.constants import map_dimensions

import json
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import datetime, timedelta
import glob
import config

In [2]:
api_key = config.api_key
headers = config.headers

In [3]:
today = datetime.now() - timedelta(1)
date = today.strftime('%Y-%m-%d')
date

'2024-02-17'

In [4]:
flist = glob.glob(f'data/{date}/*_smp_data.json')
flist

['data/2024-02-17/2024-02-17_smp_data.json']

In [5]:
## save data into json file
sample_file_path = flist[0]

In [6]:
with open(sample_file_path, 'r') as f:
    sample_data = json.load(f)

In [7]:
match_list = sample_data['data']['relationships']['matches']['data']
len(match_list), match_list[:5]

(317,
 [{'type': 'match', 'id': '762369d9-fd81-4b5a-9bd7-3aca6428a64b'},
  {'type': 'match', 'id': '1c7336ad-ab2e-43f5-8a15-4fdfffbee81c'},
  {'type': 'match', 'id': '023492b1-60b2-4c10-9a9c-3dde01168daa'},
  {'type': 'match', 'id': '4b997684-47d7-4c9a-9173-dc3fd26d75fd'},
  {'type': 'match', 'id': '2e6da0e6-effa-4331-864f-8ed4b8bad683'}])

### Role death record data

In [8]:
def timestamp_into_sec(t):
    hrs = int(t.split('T')[-1].split(':')[0])
    min = int(t.split('T')[-1].split(':')[1])
    sec = int(t.split('T')[-1].split(':')[2][:2])
    mili = int(t.split('T')[-1].split(':')[2][3:-1])
    if len(str(mili)) == 3:
        ans = sec + min*60 + mili/1000 + hrs*60*60
    else:
        ans = sec + min*60 + mili/(10**len(str(mili))) + hrs*60*60
    return ans

In [9]:
features_D = ['killer_name', 'killer_id', 'killer_loc_x','killer_loc_y',
              'victim_name', 'victim_id', 'victim_loc_x', 'victim_loc_y',
              'killed_by', 'map', 'match_id', 'time', 'time_pass', 'phase']
log_df = pd.DataFrame(columns = features_D)
log_df

,killer_name,killer_id,killer_loc_x,killer_loc_y,victim_name,victim_id,victim_loc_x,victim_loc_y,killed_by,map,match_id,time,time_pass,phase


In [10]:
pubg = PUBG(api_key, shard = 'kakao')

In [11]:
error_list = []
for i, match in tqdm(enumerate(match_list)):
    try:
        match_id = match['id']
        current_match = pubg.match(match_id)
        telemetry = current_match.get_telemetry()
        st = telemetry.started()
        
        ## get phase info
        circles = telemetry.circle_positions()
        whites = np.array(circles['white'])
        phases = np.where(whites[1:, 4] - whites[:-1, 4] != 0)[0] + 1
        
        phase_time = []
        for phase in phases:
            phase_time.append(whites[phase][0])
        
        ## get data    
        map = telemetry.map_name()
        kill_log = telemetry.filter_by(event_type='log_player_kill_v2')
    
        for j, log in enumerate(kill_log):
            try: 
                kname = log['killer']['name']
                kid = log['killer']['account_id']
                kloc = [log['killer']['location']['x'], log['killer']['location']['y']]
                vname = log['victim']['name']
                vid = log['victim']['account_id']
                vloc = [log['victim']['location']['x'], log['victim']['location']['y']]
                weap = log['d_bno_damage_info']['damage_causer_name']
                time = log.timestamp
                time_pass = timestamp_into_sec(time) - timestamp_into_sec(st)
                phase = sum([time_pass <= x for x in phase_time])
        
                log_df.loc[len(log_df)] = [kname, kid, kloc[0], kloc[1],
                                           vname, vid, vloc[0], vloc[1],
                                           weap, map, match_id, time, time_pass, phase]
            except:
                continue
    except:
        error_list.append(match_id)
        print(match_id)
        continue

317it [30:15,  5.73s/it]


In [12]:
log_df.shape, log_df.head()

((19676, 14),
     killer_name                                 killer_id  killer_loc_x  \
 0    AI-___-Bot  account.84f938d75eba4fc3b62368a19011db77  270148.09375   
 1      XVII_V17  account.4bdab5cdd7c74ed28ef2ccb8eb18c994  364360.46875   
 2      gjdudtod  account.b8efac09d9bc428ba91256ede8873c36  495162.62500   
 3      M_ARIN_E  account.509614ee71864be0979a7a3116f9e034  273302.25000   
 4  fluffy_punch  account.2f5bb347c84b477cad44a53f69d734f3  491784.09375   
 
    killer_loc_y  victim_name                                 victim_id  \
 0   426676.6250  01088076393  account.f2d2fe08fd214178ad6447dc098190e9   
 1   538774.8750    B_LostArk  account.c4d78e8d2bdb45debc9d6fb941afccf2   
 2   403831.3750      H-0508Y  account.29861a08455f40029d8c45d35d14c361   
 3   445791.1875  ponkeycandy  account.d3dd22b5b19148219c1e752384de0550   
 4   398321.3125      G-0508E  account.5c9ce5f5418b4baab45e360d6daba021   
 
    victim_loc_x  victim_loc_y         killed_by      map  \
 0  269077.0312

In [13]:
log_df.head()

,killer_name,killer_id,killer_loc_x,killer_loc_y,victim_name,victim_id,victim_loc_x,victim_loc_y,killed_by,map,match_id,time,time_pass,phase
0,AI-___-Bot,account.84f938d75eba4fc3b62368a19011db77,270148.09375,426676.6250,01088076393,account.f2d2fe08fd214178ad6447dc098190e9,269077.03125,425554.31250,WeapM16A4_C,Vikendi,762369d9-fd81-4b5a-9bd7-3aca6428a64b,2024-02-16T23:42:27.635Z,95.49546,8
1,XVII_V17,account.4bdab5cdd7c74ed28ef2ccb8eb18c994,364360.46875,538774.8750,B_LostArk,account.c4d78e8d2bdb45debc9d6fb941afccf2,365190.71875,538075.12500,WeapAUG_C,Vikendi,762369d9-fd81-4b5a-9bd7-3aca6428a64b,2024-02-16T23:42:41.177Z,109.03746,8
2,gjdudtod,account.b8efac09d9bc428ba91256ede8873c36,495162.62500,403831.3750,H-0508Y,account.29861a08455f40029d8c45d35d14c361,495606.96875,403759.71875,WeapBerreta686_C,Vikendi,762369d9-fd81-4b5a-9bd7-3aca6428a64b,2024-02-16T23:43:08.937Z,136.79746,8
3,M_ARIN_E,account.509614ee71864be0979a7a3116f9e034,273302.25000,445791.1875,ponkeycandy,account.d3dd22b5b19148219c1e752384de0550,273061.90625,445517.03125,WeapM249_C,Vikendi,762369d9-fd81-4b5a-9bd7-3aca6428a64b,2024-02-16T23:43:17.710Z,145.57046,8
4,fluffy_punch,account.2f5bb347c84b477cad44a53f69d734f3,491784.09375,398321.3125,G-0508E,account.5c9ce5f5418b4baab45e360d6daba021,491985.09375,402450.68750,WeapFNFal_C,Vikendi,762369d9-fd81-4b5a-9bd7-3aca6428a64b,2024-02-16T23:43:20.517Z,148.37746,8


### save the match log data

In [ ]:
json_data = log_df.to_json(orient='records')
len(json_data) #json_data

In [15]:
json_file_name = f'data/{date}/{date}_log_data.json'
json_file_name

'data/2024-02-17/2024-02-17_log_data.json'

In [16]:
with open(json_file_name, 'w') as f:
    json.dump(json_data, f)